<a href="https://colab.research.google.com/github/fukuta-takenaka/pub_colab/blob/main/NDB_open_data_excel2csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install --upgrade openpyxl
from io import BytesIO
import openpyxl
import requests
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
url = 'https://www.mhlw.go.jp/content/12400000/000821450.xlsx'
r = requests.get(url, allow_redirects=True)
wb = openpyxl.open(BytesIO(r.content))

In [3]:
ws = wb.worksheets[0]
ws['A1'].value

'診療年月：H31年04月～R02年03月\u3000D 検査\u3000外来\u3000※集計結果が10未満の場合は「‐」で表示（10未満の箇所が1箇所の場合は10以上の最小値を全て「‐」で表示）※集計対象期間内に名称や点数・金額等に変更がある場合、集計対象期間当初の情報で表示'

In [4]:
data_cells = ws['A5:BA1319']
cols = [cell.value.replace('\n', '') for cell in ws['A3:F3'][0] + ws['G4:BA4'][0]]

In [5]:
data = list()
tmp_cls_code = ''
tmp_cls_name = ''
for row in data_cells:
  if row[0].value:
    tmp_cls_code = row[0].value
    tmp_cls_name = row[1].value
  data.append([tmp_cls_code, tmp_cls_name] + [cell.value for cell in row[2:]])

In [20]:
df = pd.DataFrame(data, columns=cols)
df.head()

,分類コード,分類名称,診療行為コード,診療行為,点数,総計,北海道,青森県,岩手県,宮城県,...,愛媛県,高知県,福岡県,佐賀県,長崎県,熊本県,大分県,宮崎県,鹿児島県,沖縄県
0,D000,尿中一般物質定性半定量検査,160000310,尿中一般物質定性半定量検査,26,67663339,3715559,847707,877458,1600628,...,862351,545020,2807674,481349,851405,1152042,889535,823475,1117181,702541
1,D001,尿中特殊物質定性定量検査,160000410,尿蛋白,7,2094306,133410,19438,26094,43524,...,23826,13834,61385,13078,31446,35508,27562,13211,42602,32850
2,D001,尿中特殊物質定性定量検査,160001310,ＶＭＡ定性（尿）,9,2234,61,213,159,68,...,-,-,31,-,-,-,-,-,70,-
3,D001,尿中特殊物質定性定量検査,160001410,Ｂｅｎｃｅ Ｊｏｎｅｓ蛋白定性（尿）,9,11531,343,66,91,117,...,91,64,307,150,47,130,28,84,25,149
4,D001,尿中特殊物質定性定量検査,160001710,尿グルコース,9,491982,75355,785,354,6259,...,449,77,8090,3952,6375,2158,502,1749,501,103


In [21]:
df.to_csv('第6回NDBオープンデータ_都道府県別算定回数.csv', index=False)